# 특징 유형이 다른 경우의 특징 선택

In [1]:
import os
import pandas as pd

# 불필요한 경고 표시 생략
import warnings
warnings.filterwarnings(action = 'ignore')

a=%pwd # 현재 경로 a에 할당
os.chdir(a) # 파일 로드 경로 설정

# 데이터 로드 및 모델 성능 확인

특징에 연속형 & 이진형이 섞여있는 샘플 데이터

In [2]:
df = pd.read_csv("australian.csv")
df.head()

,A1,A2,A3,A4,A5,A6,A7,A8,A9,A10,A11,A12,A13,A14,Class
0,1,2208,1146,2,4,4,1585,0,0,0,1,2,100,1213,0
1,0,2267,7,2,8,4,165,0,0,0,0,2,160,1,0
2,0,2958,175,1,4,4,125,0,0,0,1,2,280,1,0
3,0,2167,115,1,5,3,0,1,1,11,1,2,0,1,1
4,1,2017,817,2,6,4,196,1,1,14,0,2,60,159,1


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 690 entries, 0 to 689
Data columns (total 15 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   A1      690 non-null    int64
 1    A2     690 non-null    int64
 2    A3     690 non-null    int64
 3    A4     690 non-null    int64
 4    A5     690 non-null    int64
 5    A6     690 non-null    int64
 6    A7     690 non-null    int64
 7    A8     690 non-null    int64
 8    A9     690 non-null    int64
 9    A10    690 non-null    int64
 10   A11    690 non-null    int64
 11   A12    690 non-null    int64
 12   A13    690 non-null    int64
 13   A14    690 non-null    int64
 14  Class   690 non-null    int64
dtypes: int64(15)
memory usage: 81.0 KB


In [4]:
# 특징과 라벨 분리
X = df.drop('Class', axis = 1)
Y = df['Class']

In [5]:
# 학습 데이터와 평가 데이터 분리
from sklearn.model_selection import train_test_split
Train_X, Test_X, Train_Y, Test_Y = train_test_split(X, Y)

In [6]:
Train_X.shape # 샘플(517) 대비 특징(14)이 약간 더 많은 정도

(517, 14)

## 모델 성능 측정 (모든 특징 사용)

In [7]:
# 특징 선택 전 성능 확인 with KNN
from sklearn.neighbors import KNeighborsClassifier as KNN
model = KNN().fit(Train_X, Train_Y)
pred_Y = model.predict(Test_X)

from sklearn.metrics import f1_score
print(f1_score(Test_Y, pred_Y))

0.6013071895424835


연속형 & 이진형 특징 구분

In [8]:
# 유니크한 값의 개수를 바탕으로 연속형과 이진형 변수 구분
# 실제로는 dtype / 상태공간을 확인할 필요 있음

continuous_cols = [col for col in Train_X.columns if len(Train_X[col].unique()) > 3] # unique값이 3개이상인 feature
binary_cols = [col for col in Train_X.columns if len(Train_X[col].unique()) <= 3] # unique값이 3개이거나 그 이하인 feature

![image](https://user-images.githubusercontent.com/74717033/134632384-fffb3524-6c0c-47bb-ac8c-7cc200cbae4b.png)

In [9]:
# 연속형 변수에 대해서는 f_classif을, 이진형 변수에 대해서는 chi2를 적용
from sklearn.feature_selection import *


# 서로 다른 두 통계량 기준을 비교하기 어려움 -> p-value로 측정
# f_regression(X, Y) => 결과값 : (statistics, p-value) 
continous_cols_pvals = f_classif(Train_X[continuous_cols], Train_Y)[1] #연속형은 f_classif, [1]이 p-value
binary_cols_pvals = chi2(Train_X[binary_cols], Train_Y)[1] # 이진형은 chi2, [1]이 p-value

In [10]:
# 각각을 Series로 변환 (value: pvalue, index: colum name)
cont_pvals = pd.Series(continous_cols_pvals, index = continuous_cols)
binary_pvals = pd.Series(binary_cols_pvals, index = binary_cols)

In [11]:
# cont_pvals과 binary_pvals을 concat
pvals = pd.concat([cont_pvals, binary_pvals])
pvals.sort_values(ascending = True, inplace = True) # 오름차순으로 정렬 (앞에 나올수록 좋은 특징->p-value)

In [12]:
# 특징 선택 
k = 10

# p-value를 k값 의 개수 만큼 (1~10위) 만 추출
s_Train_X = Train_X[pvals.iloc[:k].index]  
s_Test_X = Test_X[pvals.iloc[:k].index] # 결국 필요한 것은 컬럼명! -> index로 가져옴

확인

In [13]:
pvals.iloc[:1].index

Index([' A8'], dtype='object')

In [14]:
Train_X[pvals.iloc[:1].index]

,A8
486,1
450,0
487,1
47,0
451,1
...,...
544,1
620,0
600,0
402,0


In [15]:
# 특징 선택 후 성능 확인
from sklearn.neighbors import KNeighborsClassifier as KNN
model = KNN().fit(s_Train_X, Train_Y)
pred_Y = model.predict(s_Test_X)

from sklearn.metrics import f1_score
print(f1_score(Test_Y, pred_Y))

0.5906040268456376
